<a href="https://colab.research.google.com/github/e-s-23/Prog2kakushin/blob/main/work3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#画像処理を用いたプログラム
現時点では  
1.ペイントソフトなどで作成した画像を使用するプログラム  
2.スクレピングなどで拾ってきた画像を扱うプログラム  
を考えている。